# ICC305 Avaliação de Desempenho - 2023/1

## Atividade 3

**Giovanna** Andrade Santos - 22052563

**Marcos** Avner Pimenta de Lima - 21852448

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import math
import time

from scipy.stats import chisquare
from scipy.stats import chi2_contingency

from ad_library import myfunctions
from collections import defaultdict

## Parte 1

In [2]:
#nº de clientes
n1 = 10 ** 3
n2 = 10 ** 6
n3 = 10 ** 9
n4 = 10 ** 12

#taxa de entrada --> 9 clientes/s
tc = 9

#taxa de serviço --> 10 clientes/s
ts = 10

# https://introcs.cs.princeton.edu/python/43stack/mm1queue.py

In [14]:
# Accept float command-line arguments lamb and mu. Simulate an
# M/M/1 queue with arrival rate lamb and service rate mu.
def mm1(n=1000, tc=9., ts=10., gerador=myfunctions.G5RandomGenerator()):
    lamb = 1. / tc
    mu = 1. / ts
    fila = list() # fila de espera
    # espera = np.zeros(n)
    # o vetor de tempos de espera irá armazenar valores em milisegundos (inteiros)
    # isso é para economizar memória RAM
    espera = np.zeros(n, dtype='uint16')
    cont = 0
    
    prox_chegada = myfunctions.va_exp(lamb, gerador) # computa o tempo da proxima chegada.
    prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador) # computa o tempo da conclusao do proximo atendimento.
    
    # inicia a simulação da fila MM1.
    while n > 0:

        while prox_chegada < prox_atend:
            # simula a proxima chegada
            fila.append(prox_chegada)
            prox_chegada += myfunctions.va_exp(lamb, gerador)

        # Next event is a service completion.
        if fila:
            chegada = fila.pop(0)
            # para redução do consumo de RAM os tempos de espera são convertidos para milisegundos (inteiros)
            espera[cont] = int((prox_atend - chegada) * 1000) 
            # espera[cont] = prox_atend - chegada
            cont += 1

        # atualiza a fila de espera.
        if fila:
            # se a fila não estiver vazia, o termino do proximo atendimento 
            # é incrementando com o ts
            prox_atend += myfunctions.va_exp(mu, gerador)
        else:
            # se a fila estiver vazia, o termino do próximo atendimento
            # é incrementando com o (tc + ts)
            prox_atend = prox_chegada + myfunctions.va_exp(mu, gerador)
        
        n -= 1
    return espera

In [6]:
#função para calcular intervalo de confiança
def ic(n, media, values: list, alpha=.05):
    std = np.std(values)
    H = st.t.ppf(q=1. - alpha / 2., df=n-1) * std
    li = media - H
    ls = media + H
    return (li, ls)

In [18]:
g5 = myfunctions.G5RandomGenerator()
g5.reset() # reinicia o gerador com a seed para replicação de resultados

for n in [n1, n2, n3]:    
    print(f'Iniciando simulação para n={n}')
    start = time.time()
    tot_espera = mm1(n, tc, ts, g5)
    end = time.time()
    print(f'Fim da simulação!')
    print(f'Tempo: {end-start} ms')
    
    print('Salvando tempos de espera')
    with open(f'mm1_n{n}', 'wb') as f:
        np.save(f, tot_espera)
        
    print('Calculando intervalo de confiança')
    soma = np.sum(tot_espera)
    media = myfunctions.mean(tot_espera)
    inter_conf = ic(n, media, tot_espera)
    
    print(f'N: {n}')
    print(f'Espera total: {soma} ms')
    print(f'Espera Média: {media} ms')
    print(f'Espera IC: {inter_conf} ms')
    print()

Iniciando simulação para n=1000
Fim da simulação!
Tempo: 0.055075883865356445ms
Salvando tempos de espera
Calculando intervalo de confiança
N: 1000
Espera total: 673838 ms
Espera Média: 673.838 ms
Espera IC: (-396.5929554419736, 1744.2689554419735) ms

Iniciando simulação para n=1000000
Fim da simulação!
Tempo: 0.9812638759613037ms
Salvando tempos de espera
Calculando intervalo de confiança
N: 1000000
Espera total: 1003611329 ms
Espera Média: 1003.611329 ms
Espera IC: (-961.434759208849, 2968.657417208849) ms

Iniciando simulação para n=1000000000
Fim da simulação!
Tempo: 1006.5161719322205ms
Salvando tempos de espera
Calculando intervalo de confiança
N: 1000000000
Espera total: 1000629351149 ms
Espera Média: 1000.629351149 ms
Espera IC: (-964.2031208617949, 2965.4618231597947) ms



In [23]:
valor_esperado = myfunctions.valor_esperado(tc, ts)
print(f'O valor esperado do tempo de espera para os valores de lambda e mi é {valor_esperado:.5} segundos')

O valor esperado do tempo de espera para os valores de lambda e mi é 0.9 segundos


O valor médio esperado é de aproximadamente `0.9 segundos`.

Para `n=1000` o tempo médio $\overline{X}(n)$ foi de aproximadamente `0.673 segundos` bem longe do valor esperado, embora o intervalo de confiança `[-0.396, 1.733]`  contenha o valor esperado.

Para `n=1000000` o tempo médio $\overline{X}(n)$ foi de aproximadamente `1.003 segundos`, mais próximo do valor esperado, e o intervalo de confiança `[-0.961, 2.968]` também contem o valor esperado.

Para `n=1000000000` o tempo médio $\overline{X}(n)$ foi de aproximadamente `1.000 segundos`, um pouco mais próximo do valor esperado, e novamente intervalo de confiança `[-0.961, 2.968]` contem o valor esperado.

## Parte 2

Intervalo de confiança é (-0.3966741950472672, 1.74532996122303)
